# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Cloud Storage

下記コードでGCPに接続

In [2]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## Google Sheets

ライブラリのインストール

In [5]:
!pip install --upgrade -q gspread

GoogleSheetsの認証

In [6]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## ESTAT_APP_ID

In [3]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [4]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

'statsDataId'に統計表コードを指定

In [ ]:
statsDataId = '0000020201'

メタ情報を取得する関数を作成

In [ ]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## DataFrameの作成

メタ情報をDataFrameに整形する

In [ ]:
def get_cards_dataframe(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)
 
  # STAT_NAME
  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # cat01
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # DataFrame
  df = pd.json_normalize(cat01)[['@code','@name']]
  
  # 統計表情報の追加
  df['statsDataId'] = statsDataId
  df['statsDataName'] = STAT_NAME

  # 列名の変更
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df= df.rename(columns=columns)

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)
  
  
  # 並べ替え
  df = df.reindex(columns=['statsDataId', 'statsDataName', 'categoryCode','categoryName'])
  
  return df


# print(len(category_codes))

In [ ]:
df = get_cards_dataframe(statsDataId)
df

## 統計カード管理情報をSpreadSheetで加工

統計カードの管理情報を SpreadSheetで管理する

前セルで作業したDataFrameをcsvでコピーするなどして作業すること


[statistics-hyogo/統計カード管理](https://docs.google.com/spreadsheets/d/1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4/edit#gid=0)

# コンテンツ作成

## 統計分野一覧

GoogleSheetsから統計分野一覧を取得

In [13]:
def get_statistics_fields():

  # シート情報の設定
  id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
  sheet_name = 'statisticsFields'
  worksheet = gc.open_by_key(id).worksheet(sheet_name)
  
  return worksheet.get_all_records()


In [ ]:
statistics_fields = get_statistics_fields()
print(statistics_fields)

GoogleDriveとGoogleCloudStorageに保存

# BigQueryへ保存

ライブラリのインストール

In [ ]:
!pip install --upgrade -q gspread

SpreadSheetの認証

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheetを読み込んでDataFrameに変換

In [ ]:
# スプレッドシートの情報
ss_id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("シート1")

# DataFrameに格納
dic = worksheet.get_all_records(head=1)
df = pd.DataFrame(dic)
df

pandas-gbqのインストール

In [ ]:
!pip install pandas-gbq

DataFrameをBigQueryへ保存

In [ ]:
import pandas as pd

# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'contents'
table_id='cards'

# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")